In [1]:
from bs4 import BeautifulSoup
import numpy as np

In [23]:
def getSoup(file):
  with open(file, "r") as infile:
    xml_string = infile.read()
  soup = BeautifulSoup(xml_string, "xml")
  #trace_strings = soup.find_all("trace")
  return soup

In [3]:
def arrayToPoints (arr) :
  points = []
  for i in arr:
    points.append(point(i[0], i[1]))
  return(points)

In [24]:
import numpy as np
from rdp import rdp
import math
EPSILON = 1e-12
MACHINE_EPSILON = 1.12e-16


class point:
  def __init__(self, x, y):
    self.x = x
    self.y = y
#note calculations may be off because of int vs float
def isMachineZero (val) : 
  return (val >= -MACHINE_EPSILON and val <= MACHINE_EPSILON)
def hypot (x, y) : 
  return math.sqrt(x * x + y * y)
#def point (x, y) : return ( Point(x,y) )
def pointLength (p) : 
  return hypot(p.x, p.y)
def pointNegate (p) : 
  return point(-p.x, -p.y)
def pointAdd (p1, p2) : 
  return point(p1.x + p2.x, p1.y + p2.y)
def pointSubtract (p1, p2) : 
  return point(p1.x - p2.x, p1.y - p2.y)
def pointMultiplyScalar (p, n) : 
  return point(p.x * n, p.y * n)
def pointDot (p1, p2) : 
  return p1.x * p2.x + p1.y * p2.y
def pointDistance (p1, p2) : 
  return hypot(p1.x - p2.x, p1.y - p2.y)
def pointNormalize (p, length = 1) : 
  return pointMultiplyScalar(p, (length / pointLength(p)) if pointLength(p) else 0) #POSSIBLE FLAW
class createSegment: 
  def __init__(self, p, i, o):
    self.p=p
    self.i=i
    self.o=o #POSSIBLE FLAW (CHANGED)

def chordLengthParameterize (points, first, last)  :
    u = [0]
    for i in range(first+1, last+1):
    #for (let i = first + 1 i <= last i++) :
        #u[i - first] = u[i - first - 1] + pointDistance(points[i], points[i - 1])
        u.append(u[i - first - 1] + pointDistance(points[i], points[i - 1]))
    
    for i in range(1, last-first+1):
    #for (let i = 1, m = last - first i <= m i++) :
        u[i] /= u[last-first]
    
    return u

# Use Newton-Raphson iteration to find better root.
def findRoot (curve, point, u) :
    curve1 = []
    curve2 = []
    # Generate control vertices for Q'
    for i in range(3) : #FLAG
        curve1.append(pointMultiplyScalar(pointSubtract(curve[i + 1], curve[i]), 3))
    
    # Generate control vertices for Q''
    for i in range(2):
    #for (let i = 0 i <= 1 i++) :
        curve2.append(pointMultiplyScalar(pointSubtract(curve1[i + 1], curve1[i]), 2))
    
    # Compute Q(u), Q'(u) and Q''(u)
    pt = evaluate(3, curve, u)
    pt1 = evaluate(2, curve1, u)
    pt2 = evaluate(1, curve2, u)
    diff = pointSubtract(pt, point)
    df = pointDot(pt1, pt1) + pointDot(diff, pt2)
    # u = u - f(u) / f'(u)
    #return isMachineZero(df) ? u : u - pointDot(diff, pt1) / df
    return (u if isMachineZero(df) else (u-pointDot(diff,pt1)/df))

# Evaluate a bezier curve at a particular parameter value
def evaluate (degree, curve, t) :
    # Copy array
    tmp = curve[:]#.slice() FLAG
    # Triangle computation
    for i in range(1, degree+1):
    #for (let i = 1 i <= degree i++) :
        for j in range(degree):
        #for (let j = 0 j <= degree - i j++) :
            tmp[j] = pointAdd(pointMultiplyScalar(tmp[j], 1 - t), pointMultiplyScalar(tmp[j + 1], t))

    return tmp[0]

def addCurve (segments, curve)  :
    prev = segments[len(segments) - 1]
    prev.o = pointSubtract(curve[1], curve[0])
    segments.append(createSegment(curve[3], pointSubtract(curve[2], curve[3]), None))

# Use least-squares method to find Bezier control points for region.
def generateBezier (points, first, last, uPrime, tan1, tan2)  :
    epsilon = EPSILON
    #abs = Math.abs
    pt1 = points[first]
    pt2 = points[last]
    # Create the C and X matrices
    C = [[0, 0],[0, 0]]
    X = [0, 0]
    for i in range(0, last-first+1):
    #for (let i = 0, l = last - first + 1 i < l i++) :
        u = uPrime[i] 
        t = 1 - u
        b = 3 * u * t
        b0 = t * t * t
        b1 = b * t
        b2 = b * u
        b3 = u * u * u
        a1 = pointNormalize(tan1, b1)
        a2 = pointNormalize(tan2, b2)
        tmp = pointSubtract(pointSubtract(points[first + i], pointMultiplyScalar(pt1, b0 + b1)), pointMultiplyScalar(pt2, b2 + b3))
        C[0][0] += pointDot(a1, a1)
        C[0][1] += pointDot(a1, a2)
        # C[1][0] += a1.dot(a2)
        C[1][0] = C[0][1]
        C[1][1] += pointDot(a2, a2)
        X[0] += pointDot(a1, tmp)
        X[1] += pointDot(a2, tmp)
    
    # Compute the determinants of C and X
    detC0C1 = C[0][0] * C[1][1] - C[1][0] * C[0][1]
    alpha1=None
    alpha2=None
    if (abs(detC0C1) > epsilon) :
        # Kramer's rule
        detC0X = C[0][0] * X[1] - C[1][0] * X[0]
        detXC1 = X[0] * C[1][1] - X[1] * C[0][1]
        # Derive alpha values
        alpha1 = detXC1 / detC0C1
        alpha2 = detC0X / detC0C1
    
    else :
        # Matrix is under-determined, try assuming alpha1 == alpha2
        c0 = C[0][0] + C[0][1]
        c1 = C[1][0] + C[1][1]
        #alpha1 = alpha2 = abs(c0) > epsilon ? X[0] / c0 : abs(c1) > epsilon ? X[1] / c1 : 0
        alpha1 = alpha2 = X[0] / c0 if abs(c0) > epsilon else X[1] / c1 if abs(c1) > epsilon else 0
    
    # If alpha negative, use the Wu/Barsky heuristic (see text)
    # (if alpha is 0, you get coincident control points that lead to
    # divide by zero in any subsequent NewtonRaphsonRootFind() call.
    segLength = pointDistance(pt2, pt1)
    eps = epsilon * segLength
    handle1 = None
    handle2 = None
    if (alpha1 < eps or alpha2 < eps) :
        # fall back on standard (probably inaccurate) formula,
        # and subdivide further if needed.
        alpha1 = alpha2 = segLength / 3
    
    else :
        # Check if the found control points are in the right order when
        # projected onto the line through pt1 and pt2.
        line = pointSubtract(pt2, pt1)
        # Control points 1 and 2 are positioned an alpha distance out
        # on the tangent vectors, left and right, respectively
        handle1 = pointNormalize(tan1, alpha1)
        handle2 = pointNormalize(tan2, alpha2)
        if (pointDot(handle1, line) - pointDot(handle2, line) > segLength * segLength) :
            # Fall back to the Wu/Barsky heuristic above.
            alpha1 = alpha2 = segLength / 3
            handle1 = handle2 = None # Force recalculation
        
    
    # First and last control points of the Bezier curve are
    # positioned exactly at the first and last data points
    return [pt1, pointAdd(pt1, handle1 or pointNormalize(tan1, alpha1)), pointAdd(pt2, handle2 or pointNormalize(tan2, alpha2)), pt2]

# Given set of points and their parameterization, try to find
# a better parameterization.
def reparameterize (points, first, last, u, curve) :
    #for (let i = first i <= last i++) :
    for i in range(first, last+1):
        u[i - first] = findRoot(curve, points[i], u[i - first])
    
    # Detect if the new parameterization has reordered the points.
    # In that case, we would fit the points of the path in the wrong order.
    #for (let i = 1, l = u.length i < l i++) :
    for i in range (1, len(u)):
        if (u[i] <= u[i - 1]):
            return False
    
    return True


class error:
  def __init__(self, error, index):
    self.error = error
    self.index = index
# Find the maximum squared distance of digitized points to fitted curve.
def findMaxError (points, first, last, curve, u)  :
    index = math.floor((last - first + 1) / 2)
    maxDist = 0
    #for (let i = first + 1 i < last i++) :
    for i in range (first+1, last):
        P = evaluate(3, curve, u[i - first])
        v = pointSubtract(P, points[i])
        dist = v.x * v.x + v.y * v.y # squared
        if (dist >= maxDist) :
            maxDist = dist
            index = i
        
    return error(maxDist, index)
    #return :
    #   error: maxDist,
    #    index: index,
    



def fit (points, closed, error) :
    #points=filterRepeats(points)
    #points=rdpFilter(points,0.3)
    length = len(points)
    if length == 0 and type(length) == int:
        return []
    segments = [createSegment(points[0], None, None)]
    if length == 1:
      return segments
    fitCubic(points, segments, error, 0, length - 1, 
    pointSubtract(points[1], points[0]), 
    pointSubtract(points[length - 2], points[length - 1]))
    if (closed) :
        segments.pop(0)
        segments.pop()
    
    return segments
#note MAY HAVE TO FIND ALTERNATIVE TO === instead of using ==
def fitCubic (points, segments, error, first, last, tan1, tan2):
    #  Use heuristic if region only has two points in it
    if (last - first == 1) :
        pt1 = points[first]
        pt2 = points[last]
        dist = pointDistance(pt1, pt2) / 3
        addCurve(segments, [pt1, pointAdd(pt1, pointNormalize(tan1, dist)), pointAdd(pt2, pointNormalize(tan2, dist)), pt2])
        return
    
    # Parameterize points, and attempt to fit curve
    uPrime = chordLengthParameterize(points, first, last)
    maxError = max(error, error * error)
    split=None
    parametersInOrder = True
    # Try not 4 but 5 iterations
    #for (let i = 0 i <= 4 i++) :
    for i in range(5):
        curve = generateBezier(points, first, last, uPrime, tan1, tan2)
        #  Find max deviation of points to fitted curve
        Max = findMaxError(points, first, last, curve, uPrime)
        if (Max.error < error and parametersInOrder) :
            addCurve(segments, curve)
            #for segment in segments:
            #  print((str(segment.i.x) if segment.i else "None") + " " + (str(segment.i.y) if segment.i else "None") + " " + (str(segment.o.x) if segment.o else "None") + " " + (str(segment.o.y) if segment.o else "None") + " " + (str(segment.p.x) if segment.p else "None") + " " + (str(segment.p.y) if segment.p else "None"))
            return
        
        split = Max.index
        # If error not too large, try reparameterization and iteration
        if (Max.error >= maxError):
            break
        parametersInOrder = reparameterize(points, first, last, uPrime, curve)
        maxError = Max.error
    
    # Fitting failed -- split at max error point and fit recursively
    tanCenter = pointSubtract(points[split - 1], points[split + 1])
    fitCubic(points, segments, error, first, split, tan1, tanCenter)
    fitCubic(points, segments, error, split, last, pointNegate(tanCenter), tan2)


def filterRepeats(points):
  filtered_points=[]
  for i in range(len(points)-1):
    if points[i].x != points[i+1].x or points[i].y != points[i+1].y:
      filtered_points.append(points[i])
  filtered_points.append(points[len(points)-1])
  return filtered_points
def rdpFilter(points, epsilon):
  arr=[]
  for i in range(len(points)):
    arr.append([points[i].x,points[i].y])
  return arrayToPoints(rdp(arr, epsilon=epsilon))


# Assign parameter values to digitized points
# using relative distances between points.

def svgPath (segments, closed, precision) :
    length = len(segments)
    precisionMultiplier = 10 ** precision
    def Round(n, precisionMultiplier):
      return ((round(n * precisionMultiplier) / precisionMultiplier) if precision < 16 else n)
    def formatPair (x, y):
      return (str(Round(x, precisionMultiplier)) + ',' + str(Round(y, precisionMultiplier)))
    first = True
    prevX = None
    prevY = None
    outX = None
    outY = None
    parts = []
    if length==1:
      #return [[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0]],[[segments[0].p.x,segments[0].p.y]]]
      return 'M' + str(segments[0].p.x) + "," + str(segments[0].p.y) #CHANGED
    
    if (not length):
        return ''
    for i in range(length):
    #for (let i = 0 i < length i++)
        #addSegment(segments[i])
        #if not(prevX==segments[i].p.x and prevY==segments[i].p.y):
        segmentData = []
        segment=segments[i]
        #repeat=False
        curX = segment.p.x
        curY = segment.p.y
        if (first) :
            parts.append('M' + formatPair(curX, curY))
        else :
            inX = curX + (segment.i.x if segment.i else 0)
            inY = curY + (segment.i.y if segment.i else 0)
            if (not (inX == curX and inY == curY and outX == prevX and outY == prevY)) :
                  # c = relative curveto:
                  #segmentData.extend([prevX, prevY, outX-prevX, outY-prevY, inX - prevX, inY - prevY, curX - prevX, curY - prevY])
                segmentData.extend([prevX, prevY, outX, outY, inX, inY, curX, curY])
                parts.append('c' +
                      formatPair(outX - prevX, outY - prevY) +
                      ' ' +
                      formatPair(inX - prevX, inY - prevY) +
                      ' ' +
                      formatPair(curX - prevX, curY - prevY))

        prevX = curX
        prevY = curY
        outX = curX + (segment.o.x if segment.o else 0)
        outY = curY + (segment.o.y if segment.o else 0)
        first = False

    # Close path by drawing first segment again
    #return [allSegmentData, endPoints]
    return ''.join(parts)

def getSegmentsPathData (segments, closed, precision)  :
    length = len(segments)
    precisionMultiplier = 10 ** precision
    def Round(n, precisionMultiplier):
      return ((round(n * precisionMultiplier) / precisionMultiplier) if precision < 16 else n)
    def formatPair (x, y):
      return (str(Round(x, precisionMultiplier)) + ',' + str(Round(y, precisionMultiplier)))
    first = True
    prevX = None
    prevY = None
    outX = None
    outY = None
    parts = []
    if length==1:
      return [[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0]],[[segments[0].p.x,segments[0].p.y]]]
      #return 'M' + str(segments[0].p.x) + "," + str(segments[0].p.y) #CHANGED
    
    if (not length):
        return []
    allSegmentData=[]
    endPoints = []
    for i in range(length):
    #for (let i = 0 i < length i++)
        #addSegment(segments[i])
        #if not(prevX==segments[i].p.x and prevY==segments[i].p.y):
        segmentData = []
        segment=segments[i]
        #repeat=False
        curX = segment.p.x
        curY = segment.p.y
        if (first) :
            parts.append('M' + formatPair(curX, curY))
        else :
            inX = curX + (segment.i.x if segment.i else 0)
            inY = curY + (segment.i.y if segment.i else 0)
            if (not (inX == curX and inY == curY and outX == prevX and outY == prevY)) :
                  # c = relative curveto:
                  #segmentData.extend([prevX, prevY, outX-prevX, outY-prevY, inX - prevX, inY - prevY, curX - prevX, curY - prevY])
                segmentData.extend([prevX, prevY, outX, outY, inX, inY, curX, curY])
                parts.append('c' +
                      formatPair(outX - prevX, outY - prevY) +
                      ' ' +
                      formatPair(inX - prevX, inY - prevY) +
                      ' ' +
                      formatPair(curX - prevX, curY - prevY))

                features = [segmentData[6]-segmentData[0], 
                            segmentData[7]-segmentData[1], 
                            math.sqrt((segmentData[0]-segmentData[2])**2+(segmentData[1]-segmentData[3])**2), 
                            math.sqrt((segmentData[6]-segmentData[4])**2+(segmentData[7]-segmentData[5])**2), 
                            math.atan2(segmentData[3]-segmentData[1], segmentData[2]-segmentData[0])-math.atan2(segmentData[7]-segmentData[1], segmentData[6]-segmentData[0]),
                            math.atan2(segmentData[5]-segmentData[7], segmentData[4]-segmentData[6])-math.atan2(segmentData[1]-segmentData[7], segmentData[0]-segmentData[6]), 1]
                allSegmentData.append(features)
                endPoints.append([segmentData[6],segmentData[7]])
            #else:
              #print("bruh what")

        prevX = curX
        prevY = curY
        outX = curX + (segment.o.x if segment.o else 0)
        outY = curY + (segment.o.y if segment.o else 0)
        first = False

    # Close path by drawing first segment again
    return [allSegmentData, endPoints]
    #return ''.join(parts)

class options:
  def __init__(self, closed, tolerance, precision):
    self.closed = closed
    self.tolerance = tolerance
    self.precision=precision

def f1(p):
  return point(p.x, p.y)

def f2(p):
  return point(p[0], p[1])

#def simplify(points, options) :
#  return fit(list(map((f1 if (type(points[0].x) == int or type(points[0].x) == float) else f2), points)), options.closed, options.tolerance if options.tolerance else 2.5)
#def getFeatures (points, options):

def getSvgPath (points, options) :
    if (len(points) == 0 and type(len(points))==int) :
        return ''
    return svgPath(fit(list(map(f1 if (type(points[0].x) == int or type(points[0].x) == float) else f2, points)), options.closed, options.tolerance if options.tolerance else 2.5), options.closed, options.precision if options.precision else 5)

def getFeatures (points, options)  :
    if (len(points) == 0 and type(len(points))==int) :
        return []
    return getSegmentsPathData(fit(list(map(f1 if (type(points[0].x) == int or type(points[0].x) == float) else f2, points)), options.closed, options.tolerance if options.tolerance else 2.5), options.closed, options.precision if options.precision else 5)
    #return getSegmentsPathData(fit(points.map(typeof points[0].x == 'number' ? (p) => point(p.x, p.y) : (p) => point(p[0], p[1])), options.closed, options.tolerance ?? 2.5), options.closed, options.precision ?? 5)

In [5]:
error_files=[]

In [6]:
def _parse_one_trace(trace_string):
  '''if len(trace_string.split("\n"))==3:
    trace_split = trace_string.split("\n")[1].split(",")
  else:
    trace_split = trace_string.split(">")
  try:
    trace_split = trace_string.split("\n")[1].split(",")
  except:
    print(trace_string.split("\n"))'''
  trace_split=trace_string.string
  if trace_split[0]=="\n":
    trace_split=trace_split[1:]
  if trace_split[-1]=="\n":
    trace_split=trace_split[:-1]
  trace_split=trace_split.split(",")
  a = [[float(s) for s in entry.strip().split(" ")] for entry in trace_split]
  return a


def getSvg(file, tolerance=100):
  with open(file, "r") as infile:
    xml_string = infile.read()
  soup = BeautifulSoup(xml_string, "xml")
  trace_strings = soup.find_all("trace")
  a=""
  for trace_string in trace_strings:
    path=getSvgPath(arrayToPoints(_parse_one_trace(trace_string)), options(False,tolerance,0))
    #a += path[0]
    a+=path
  return a 

def parse_traces(file, tolerance=30):
  with open(file, "r") as infile:
    xml_string = infile.read()
  soup = BeautifulSoup(xml_string, "xml")
  trace_strings = soup.find_all("trace")
  a=[]
  first = True
  prev = None
  for trace_string in trace_strings:
    try:
      path=getFeatures(arrayToPoints(_parse_one_trace(trace_string)), options(False,tolerance,0))
    except:
      print("error bezier curve: ")
      print(file)
    if not first:
      try:
        dx=path[1][len(path[1])-1][0] - prev[1][len(prev[1])-1][0]
        dy=path[1][len(path[1])-1][1]-prev[1][len(prev[1])-1][1]
      except:
        print("path:")
        print(path)
        print("trace_string:")
        print(trace_string.string)
        print("file:")
        print(file)
        error_files.append(file)
      a += [[dx, dy, math.sqrt(dx**2/9+dy**2/9), math.sqrt(dx**2/9+dy**2/9), 0.0, 0.0, 0]]
    a += path[0]
    prev = path[:]
    first=False
  return a 

In [7]:
getSvg("C:/Users/aiden/Downloads/ProjectData/LGINKML/INKMLs/2014train/130_miguel.inkml")

'M216.0,104.0c30.62545,-42.87563 16.17643,-14.35285 3.0,12.0c-2.97514,5.95029 -14.70437,4.0 -2.0,4.0c4.66667,0.0 9.33333,0.0 14.0,0.0c3.05447,0.0 11.33132,-1.0 11.0,-1.0M189.0,126.0c-11.32736,0.0 87.68895,1.0 78.0,1.0M215.0,146.0c-3.28632,-9.85896 -11.56794,48.56794 19.0,18.0M238.0,151.0c-4.94953,11.31321 -7.0,32.55464 -7.0,32.0M322.0,107.0c2.47215,-2.47215 -1.0,36.69195 -1.0,25.0M313.0,122.0c-7.77652,0.0 26.37685,-2.0 34.0,-2.0M408.0,53.0c0.0,-15.11859 0.1332,-3.97633 -17.0,38.0c-11.24843,27.55864 -7.80025,53.79899 -1.0,81.0c2.28278,9.13112 3.9088,15.1824 8.0,7.0M438.0,91.0c-8.90504,4.45252 0.86629,-0.09469 12.0,-16.0c1.93041,-2.75773 -8.06412,21.19236 -11.0,30.0c-1.42468,4.27404 -1.63067,7.0 -8.0,7.0c-9.66667,0.0 38.66667,0.0 29.0,0.0M407.0,123.0c-21.23067,0.0 88.90153,0.0 87.0,0.0M438.0,146.0c5.06979,-10.13958 -0.29457,5.29457 -2.0,7.0c-20.04864,20.04864 9.6451,17.3549 19.0,8.0M462.0,147.0c0.0,-8.6319 -21.36691,45.36691 -14.0,38.0M506.0,51.0c0.0,-0.66667 0.0,-1.33333 0.0,-2.0M506.0,

In [8]:
error_files

[]

In [9]:
#old y data generation

from collections import defaultdict
# todo: Add something to return the sequence of unique identifiers ('x_1' etc.)
def parse_symbols_and_alignments(file):
  with open(file, "r") as f:
    string = f.read()
  #lg=[]
  rel=0 #BOOLEAN sorta
  objects=[]
  rels=defaultdict(lambda: "NoRel")
  for part in string.split("\n"):
    line=part.split(", ")
    if (line[0]=="# Relations from SRT:"):
      rel=1
    else:
      if (line[0]!=''):
        if rel:
          #if rel > 1:
          rels[(line[1], line[2])]=line[3]
          #rel+=1
          #prev=line[1]
        else:
          if (line[0][0]=="#"):
            continue
          del line[3]
          objects.append((line[1:3], [int(l) for l in line[3:]]))
  def key(obj):
    return min(obj[1])
  objects=sorted(objects, key=key)
  output=[]

  output.append(objects[0][0][1])
  prev=objects[0][0][0]
  #print(rels)
  for ob in objects[1:]:
    #print((ob[0][0], prev))
    output.append(rels[(prev, ob[0][0])])
    output.append(ob[0][1])
    prev=ob[0][0]
    #output.append(ob[0][1])
    #output.append(rels[ob[0][0]])
  '''try:
    output.append(objects[-1][0][1])
  except:
    print(file)'''
  #return(objects, rels)
  return(output)



In [9]:
class Node:
  def __init__(self, symbol, relations, parent, strokes):
    #self.id = id
    self.symbol = symbol
    self.relations = relations
    self.parent = parent
    self.strokes = strokes
  '''def get_root(self, nodes):
    node=self
    while(nodes[node].parent!=None):
      node=nodes[node].parent
    #node=Node(self.symbol,self.relations,self.parent,self.strokes)
    #while(nodes[node.parent].parent != None):
    #  node=nodes[node.parent]
    return node'''

def get_root(node, nodes):
    while(nodes[node].parent!=None):
      node=nodes[node].parent
    return node


#nodes={"A_1" : Node("A_1", "A", ["Right"], ["x_1"], None, ), }

In [10]:
from collections import defaultdict
def lg_to_tree(file):
  with open(file, "r") as f:
    string = f.read()
  rel=0 #BOOLEAN sorta
  nodes=defaultdict(lambda: Node(None, [], None, []))
  for part in string.split("\n"):
    line=part.split(", ")
    if (line[0]=="# Relations from SRT:"):
      rel=1
    else:
      if (line[0]!=''):
        if rel:
          #if rel > 1:
          nodes[line[1]].relations.append((line[2],line[3]))
          nodes[line[2]].parent=line[1]
          #rels[(line[1], line[2])]=line[3]
          #rel+=1
          #prev=line[1]
        else:
          if (line[0][0]=="#"):
            continue
          del line[3]
          nodes[line[1]].symbol=line[2]
          nodes[line[1]].strokes=line[3:]
  return(nodes)



In [11]:
length=0
def traverse_tree(tree, current, order):
  strokes=current.strokes
  relations_order={"Above":1, "Below":2, "Inside":3, "Sup":4, "Sub":5, "Right":6}
  relations=sorted(current.relations, key = lambda rel: relations_order[rel[1]])
  ord=order
  global length
  for x in range(len(strokes)):
    ord[strokes[x]]=length
    length+=1
  for i in range(len(relations)):
    ord=traverse_tree(tree, tree[relations[i][0]], ord)
  #if len(relations)==0:
  return(ord)

def reorder(file):
  tree=lg_to_tree(file)
  order = {}
  root = get_root(list(tree.keys())[0], tree)
  #for i in range(len(root.strokes)):
    #order[root.strokes[i]]=i
  global length
  length=0
  return(traverse_tree(tree, tree[root], order))

In [46]:
def convert_sequence(sequence):
    symbols=sequence[::2]
    relations=sequence[1::2]
    return(symbols, relations)


In [60]:
#\frac{9x^2 + 5}{2}

#symbols = ['9','x','2','-','2','fractionbar','5','\\cdot ','x','2','fractionbar','5']
#relations = ["Right", "Sup", "DFS", "UTF", "NTB", "BTD", "UFD", "Right", "Sub", "OST", "BTD"]

symbols, relations=convert_sequence(['1', 'NTB', '-', 'BTD', 'r', 'Sup', '2', 'DFS-UFD', '=', 'UTF', '1', 'NTB', '-', 'BTD', '(', 'Right', 'R', 'Right', '-', 'Right', 'm', 'Right', ')', 'Sup', '2', 'DFS-UFD', '+', 'UTF', '1', 'NTB', '-', 'BTD', '(', 'Right', 'R', 'Right', '+', 'Right', 'm', 'Right', ')', 'Sup', '2'])

def parse_latex(symbols, relations):
  utf=0
  open_brackets=0
  latex_string=symbols[0]
  for i in range(len(relations)):
    s=symbols[i+1]
    full_rel=relations[i]
    split_rel=full_rel.split('-')
    add_symbol=False
    print(split_rel)
    for rel in split_rel:
      if rel in ["DFS", "UFS", "OFI", "UFD"]:
        #latex_string+=("}"+s)
        latex_string+="}"
        open_brackets-=1
        add_symbol=True
      #if rel == "BTD":
        #latex_string+=(s+"}")
        #print(latex_string)
      if rel == "NTB":
        #print(latex_string)
        if utf:
          latex_string+=("}{")
          utf-=1
        else:
          latex_string="\\frac{"+latex_string+"}{"
          open_brackets+=1
      if rel == "UTF":
        latex_string+=("\\frac{"+s)
        open_brackets+=1
        utf+=1
      if rel == "Sub":
        latex_string+=("_{"+s)
        open_brackets+=1
      if rel == "Sup":
        latex_string+=("^{"+s)
        open_brackets+=1
      if rel in ["Inside", "Right", "BTD"]:
        add_symbol=True
      #  latex_string+=s
    
      '''if rel == "OST":
        if utf:
          latex_string+=("}}{")
          utf-=1
        else:
          latex_string="\\frac{"+latex_string+"}}{"
          '''
      #if rel in ["Inside", "Right"]:
        #latex_string+=s
    if add_symbol:
      latex_string+=s
  latex_string+=("}")*open_brackets
  return latex_string
  
parse_latex(symbols, relations)

['NTB']
['BTD']
['Sup']
['DFS', 'UFD']
['UTF']
['NTB']
['BTD']
['Right']
['Right']
['Right']
['Right']
['Sup']
['DFS', 'UFD']
['UTF']
['NTB']
['BTD']
['Right']
['Right']
['Right']
['Right']
['Sup']


'\\frac{1}{r^{2}}=\\frac{1}{(R-m)^{2}}+\\frac{1}{(R+m)^{2}}'

In [13]:
missing_relations=set()
missing_relations_freq=defaultdict(lambda: 0)
pop_errors=[]

In [35]:
arr=[]

In [65]:
import pickle as pkl
from collections import defaultdict
with open("C:/Users/aiden/Downloads/ProjectData/Vocab.pkl", "rb") as f:
    vocab = pkl.load(f)
last_symbol=None
new_relations=[]
new_rels=defaultdict(lambda: [])
def traverse_relations(tree, current, new_rels, incoming_rel, prev_symbol):
  relations_order={"Above":1, "Below":2, "Inside":3, "Sup":4, "Sub":5, "Right":6}
  #print(current)
  relations=sorted(tree[current].relations, key = lambda rel: relations_order[rel[1]])
  last_symbol=current
  if len(relations) > 0:
    #tree[current].symbol == "-" and 
    if relations[0][1]=="Above":
      if (prev_symbol):
        new_rels[prev_symbol].pop()
        new_rels[prev_symbol].append("UTF")
      #new_relations.append("UTF")
      #new_relations.append(relations[i][0])
      #new_relations.append("BTD")
      #new_relations.append(relations[i][0])
    #elif relations[0][1]=="Below":
    #  new_rels[]
    elif relations[0][1]=="Below":
      new_rels[current].append("LB")
    else:
      new_rels[current].append(relations[0][1])
      #new_relations.append(relations[i][1])
      #new_relations.append(relations[i][0])
    for i in range(len(relations)):
      thing=traverse_relations(tree, relations[i][0], new_rels, relations[i][1], last_symbol)#CHANGED
      last_symbol=thing[0]
      new_rels=thing[1]
      if relations[i][1]=="Above":
        new_rels[current].append("BTD")
  else:
    new_rels[current]=[]
  if incoming_rel=="Above":
    new_rels[last_symbol].append("NTB")
  elif incoming_rel=="Below":
    if tree[prev_symbol].symbol=="\lim":
      new_rels[last_symbol].append("UFL")
    else:
      new_rels[last_symbol].append("UFD")
  elif incoming_rel=="Sub":
    new_rels[last_symbol].append("UFS")
  elif incoming_rel=="Sup":
    new_rels[last_symbol].append("DFS")
  elif incoming_rel=="Inside":
    new_rels[last_symbol].append("OFI")
  #print(last_symbol)
  return (last_symbol, new_rels)

def convert_relations(file):
  #arr=[]
  tree=lg_to_tree(file)
  root = get_root(list(tree.keys())[0], tree)
  try:
    new_rels=traverse_relations(tree, root, defaultdict(lambda: []), None, None)[1]
    output=[]
    not_in_vocab=False
    for symbol in new_rels:
      print(symbol)
      print(new_rels[symbol])
      #arr.append(tree[symbol].symbol)
      output.append(vocab[tree[symbol].symbol])
      if len(new_rels[symbol]):
        relation="-".join(new_rels[symbol])
        if relation in vocab:
          output.append(vocab[relation])
          #arr.append(relation)
        else:
          print("missing: " +relation)
          missing_relations.add(relation)
          missing_relations_freq[relation]+=1
          not_in_vocab=True
  except:
    print("pop error: "+file)
    pop_errors.append(file)
    return None
  
  
  if not_in_vocab:
    return None
  #print(arr)
  return output

def get_latex(file):
  arr=[]
  tree=lg_to_tree(file)
  root = get_root(list(tree.keys())[0], tree)
  try:
    new_rels=traverse_relations(tree, root, defaultdict(lambda: []), None, None)[1]
    output=[]
    not_in_vocab=False
    for symbol in new_rels:
      arr.append(tree[symbol].symbol)
      output.append(vocab[tree[symbol].symbol])
      if len(new_rels[symbol]):
        relation="-".join(new_rels[symbol])
        if relation in vocab:
          output.append(vocab[relation])
          arr.append(relation)
        else:
          print("missing: " +relation)
          missing_relations.add(relation)
          missing_relations_freq[relation]+=1
          not_in_vocab=True
  except:
    print("pop error: "+file)
    pop_errors.append(file)
    return None
  
  
  if not_in_vocab:
    return None
  converted_sequence=convert_sequence(arr[:-1])
  return parse_latex(converted_sequence[0], converted_sequence[1])


In [66]:
get_latex("C:/Users/aiden/Downloads/ProjectData/LGINKML/LGs/2014train/80_jorge.lg")

['NTB']
['BTD']
['Sup']
['DFS', 'UFD']
['UTF']
['NTB']
['BTD']
['Right']
['Right']
['Right']
['Right']
['Sup']
['DFS', 'UFD']
['UTF']
['NTB']
['BTD']
['Right']
['Right']
['Right']
['Right']
['Sup']


'\\frac{1}{r^{2}}=\\frac{1}{(R-m)^{2}}+\\frac{1}{(R+m)^{2}}'

In [16]:
def add_to_vocab(symbol, vocab, index):
    for key in vocab:
        if vocab[key]>=index:
            vocab[key]+=1
    vocab[symbol]=index
    vocab=dict(sorted(vocab.items(), key=lambda item: item[1]))
    with open("C:/Users/aiden/Downloads/ProjectData/Vocab.pkl", "wb") as f:
        pkl.dump(vocab, f)
def add_relation_to_vocab(relation, vocab, num_relations):
    add_to_vocab(relation, vocab, num_relations)

In [18]:
$\frac{1}{r^2} = \frac{1}{(R - m)^2} + \frac{1}{(R + m)^2}$

{'Right': 0,
 'Sub': 1,
 'Sup': 2,
 'Inside': 3,
 'UTF': 4,
 'NTB': 5,
 'BTD': 6,
 'UFD': 7,
 'DFS': 8,
 'UFS': 9,
 'OFI': 10,
 'DFS-NTB': 11,
 'UFL': 12,
 'LB': 13,
 'UFS-NTB': 14,
 'DFS-UFD': 15,
 'UFS-UFD': 16,
 'DFS-DFS': 17,
 'OFI-NTB': 18,
 'OFI-UFD': 19,
 'DFS-OFI': 20,
 'UFD-DFS': 21,
 '!': 22,
 '(': 23,
 ')': 24,
 '+': 25,
 '-': 26,
 '.': 27,
 '/': 28,
 '0': 29,
 '1': 30,
 '2': 31,
 '3': 32,
 '4': 33,
 '5': 34,
 '6': 35,
 '7': 36,
 '8': 37,
 '9': 38,
 '<': 39,
 '=': 40,
 '>': 41,
 'A': 42,
 'B': 43,
 'C': 44,
 'COMMA': 45,
 'E': 46,
 'F': 47,
 'G': 48,
 'H': 49,
 'I': 50,
 'L': 51,
 'M': 52,
 'N': 53,
 'P': 54,
 'R': 55,
 'S': 56,
 'T': 57,
 'V': 58,
 'X': 59,
 'Y': 60,
 '[': 61,
 '\\Delta': 62,
 '\\alpha': 63,
 '\\beta': 64,
 '\\cos': 65,
 '\\div': 66,
 '\\exists': 67,
 '\\forall': 68,
 '\\gamma': 69,
 '\\geq': 70,
 '\\gt': 71,
 '\\in': 72,
 '\\infty': 73,
 '\\int': 74,
 '\\lambda': 75,
 '\\ldots': 76,
 '\\leq': 77,
 '\\lim': 78,
 '\\log': 79,
 '\\lt': 80,
 '\\mu': 81,
 '\\ne

In [19]:
import pickle as pkl
import os
years=["2012", "2013", "2014train", "2014test", "2016", "2019"]
#years = ["2019"]
#train=False
#name="LG"
#datapath="C:/Users/aiden/Downloads/ProjectData/"+year+"/"+ ("Train/" if train else "Test/") + name
X=[]
Y=[]
count=0
with open("C:/Users/aiden/Downloads/ProjectData/Vocab.pkl", "rb") as f:
    vocab = pkl.load(f)
for year in years:
    directory="C:/Users/aiden/Downloads/ProjectData/LGINKML/INKMLs/"+year+"/"
    for filename in os.listdir(directory):
        #the filename unincluding .inkml
        lgpath="C:/Users/aiden/Downloads/ProjectData/LGINKML/LGs/"+year+"/"+filename[:len(filename)-6]+".lg"
        if os.path.exists(lgpath):
            X.append(parse_traces(directory+filename, tolerance=25))
            LG=convert_relations(lgpath)
            if LG:
                Y.append(LG)
            else:
                print("no corresponding relation in vocab: "+lgpath)

        else:
            print("missing file:")
            print(lgpath)
        count += 1

        if count % 100 == 0: print(count)
datapath="C:/Users/aiden/Downloads/ProjectData/FeatureData/"
with open(datapath+"X_Y", "wb") as f:
    pkl.dump((X,Y), f)

error bezier curve: 
C:/Users/aiden/Downloads/ProjectData/LGINKML/INKMLs/2012/002-equation003.inkml


In [27]:
getSoup("C:/Users/aiden/Downloads/ProjectData/LGINKML/INKMLs/2014train/80_jorge.inkml")

<?xml version="1.0" encoding="utf-8"?>
<ink xmlns="http://www.w3.org/2003/InkML">
<traceFormat>
<channel name="X" type="decimal"/>
<channel name="Y" type="decimal"/>
</traceFormat>
<annotation type="category">Arithmetic</annotation>
<annotation type="expression">80</annotation>
<annotation type="UI">80_jorge</annotation>
<annotation type="writer">jorge</annotation>
<annotation type="truth">$\frac{1}{r^2} = \frac{1}{(R - m)^2} + \frac{1}{(R + m)^2}$</annotation>
<annotationXML encoding="Content-MathML" type="truth">
<math xmlns="http://www.w3.org/1998/Math/MathML">
<mrow>
<mfrac xml:id="_1">
<mn xml:id="1_1">1</mn>
<msup>
<mi xml:id="r_1">r</mi>
<mn xml:id="2_1">2</mn>
</msup>
</mfrac>
<mrow>
<mo xml:id="=_1">=</mo>
<mrow>
<mfrac xml:id="_2">
<mn xml:id="1_2">1</mn>
<mrow>
<mo xml:id="(_1">(</mo>
<mrow>
<mi xml:id="R_1">R</mi>
<mrow>
<mo xml:id="-_1">-</mo>
<mrow>
<mi xml:id="m_1">m</mi>
<msup>
<mo xml:id=")_1">)</mo>
<mn xml:id="2_2">2</mn>
</msup>
</mrow>
</mrow>
</mrow>
</mrow>
</mfr

In [37]:
convert_relations("C:/Users/aiden/Downloads/ProjectData/LGINKML/LGs/2014train/80_jorge.lg")

1_1
['NTB']
_1
['BTD']
r_1
['Sup']
2_1
['DFS', 'UFD']
=_1
['UTF']
1_2
['NTB']
_2
['BTD']
(_1
['Right']
R_1
['Right']
-_1
['Right']
m_1
['Right']
)_1
['Sup']
2_2
['DFS', 'UFD']
+_1
['UTF']
1_3
['NTB']
_3
['BTD']
(_2
['Right']
R_2
['Right']
+_2
['Right']
m_2
['Right']
)_2
['Sup']
2_3
['DFS', 'UFD']
['1', 'NTB', '-', 'BTD', 'r', 'Sup', '2', 'DFS-UFD', '=', 'UTF', '1', 'NTB', '-', 'BTD', '(', 'Right', 'R', 'Right', '-', 'Right', 'm', 'Right', ')', 'Sup', '2', 'DFS-UFD', '+', 'UTF', '1', 'NTB', '-', 'BTD', '(', 'Right', 'R', 'Right', '+', 'Right', 'm', 'Right', ')', 'Sup', '2', 'DFS-UFD']


[30,
 5,
 26,
 6,
 115,
 2,
 31,
 15,
 40,
 4,
 30,
 5,
 26,
 6,
 23,
 0,
 55,
 0,
 26,
 0,
 110,
 0,
 24,
 2,
 31,
 15,
 25,
 4,
 30,
 5,
 26,
 6,
 23,
 0,
 55,
 0,
 25,
 0,
 110,
 0,
 24,
 2,
 31,
 15]

In [ ]:
missing_relations_freq

defaultdict(<function __main__.<lambda>()>,
            {'Right-Right': 119,
             'UFD-OFI': 3,
             'Inside-Inside': 8,
             'OFI-OFI': 5,
             'UFS-DFS': 4,
             'Sup-Sup': 10,
             'UFD-UTF': 10,
             'UFD-DFS-UFD': 8,
             'UFS-UFS': 5,
             'Sub-Sub': 2,
             'UFS-OFI': 1,
             'UTF-UTF': 4,
             'BTD-BTD': 4,
             'DFS-UFD-OFI-UFD': 1,
             'UFD-DFS-NTB': 2,
             'UFD-UFD': 6,
             'UFS-UFD-OFI': 1,
             'UFS-OFI-NTB': 1,
             'UFD-NTB': 2,
             'DFS-DFS-DFS-DFS-DFS': 1,
             'UFS-UTF': 1,
             'OFI-UFD-OFI': 2,
             'UFS-DFS-NTB': 17,
             'UFS-DFS-UFD': 17})

In [ ]:
with open("C:/Users/aiden/Downloads/ProjectData/Vocab.pkl", "rb") as f:
    vocab=pkl.load(f)

In [ ]:
datapath="C:/Users/aiden/Downloads/ProjectData/FeatureData/"
with open(datapath+"X_Y_Vocab", "rb") as f:
    X, Y, vocab = pkl.load(f)

In [ ]:
length = len(X)
vocab_size = len(vocab)
X_train = []
Y_train = []
X_test = []
Y_test = []
for i in range(length):
    if (i%10):
        X_train.append(X[i])
        Y_train.append(Y[i])
    else:
        X_test.append(X[i])
        Y_test.append(Y[i])


In [ ]:
datapath="C:/Users/aiden/Downloads/ProjectData/FeatureData/"

In [ ]:
with open(datapath+"Train", "wb") as f:
    pkl.dump((X_train, Y_train, vocab), f)
with open(datapath+"Test", "wb") as f:
    pkl.dump((X_test, Y_test, vocab), f)

In [ ]:
X_train=[]
Y_train=[]

In [ ]:
with open(datapath+"Train", "rb") as f:
    X_train, Y_train, _ = pkl.load(f)
#with open(datapath+"Test", "wb") as f:
#    pkl.load((X_test, Y_test, vocab), f)

In [ ]:
print(len(X_train[5]))

25


In [ ]:
num_of_features=7
len_train=len(X_train)
len_test=len(X_test)
max_X_train_len = max([len(sample) for sample in X_train])
max_Y_train_len = max([len(sample) for sample in Y_train])
max_X_test_len = max([len(sample) for sample in X_test])
max_Y_test_len = max([len(sample) for sample in Y_test])
padding_vector=np.ones(7)
eos_index=0

NameError: name 'X_test' is not defined

In [ ]:
print(max_X_len)
print(max_Y_len)
print(len_data)

639
87
1333


In [ ]:
X_train_data_len=[len(data) for data in X_train]
Y_train_data_len=[len(data) for data in Y_train]
X_test_data_len=[len(data) for data in X_test]
Y_test_data_len=[len(data) for data in Y_test]

In [ ]:
padded_X_train=np.zeros((len_train, max_X_train_len, num_of_features))+padding_vector
padded_Y_train=np.zeros((len_train, max_Y_train_len))+eos_index
padded_X_test=np.zeros((len_test, max_X_test_len, num_of_features))+padding_vector
padded_Y_test=np.zeros((len_test, max_Y_test_len))+eos_index

In [ ]:
for i in range(len_train):
    sample_X = X_train[i]
    padded_X_train[i,:len(sample_X)]=sample_X
    sample_Y = Y_train[i]
    padded_Y_train[i,:len(sample_Y)]=sample_Y
for i in range(len_test):
    sample_X = X_test[i]
    padded_X_test[i,:len(sample_X)]=sample_X
    sample_Y = Y_test[i]
    padded_Y_test[i,:len(sample_Y)]=sample_Y

In [ ]:
print(len(X_train))
print(len(X_test))

11988
1333
